### Notebook zawiera kod z atakiem adwersalnym na sieć ResNET-18 wytrenowaną na datasecie nEMO. 
### Atak adwersalny (ang. adversarial attack) na sieć neuronową to technika, która polega na celowym modyfikowaniu danych wejściowych w taki sposób, aby wprowadzić w błąd model uczenia maszynowego. Przekształcenia danych wejściowych są często niemal niewidoczne dla człowieka.



### 1. Import bibliotek

In [ ]:
import os

import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import seaborn as sns

# from datasets import load_from_disk
# from create_data import download_and_save_dataset
from heplers.early_stopping import EarlyStopping
from heplers.augmentation import AugmentedAudioDataset
from heplers.resnet_model_definition import AudioResNet
from config import (BATCH_SIZE, NUM_EPOCHS, LEARNING_RATE, WEIGHT_DECAY,
                    DROPOUT_RATE, EARLY_STOPPING_PATIENCE, MODEL_DIR,
                    TIMESTAMP, MODEL_PATH, MAX_LENGTH, SEED, DATASET_PATH)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from datasets import load_dataset

### 2. Ładowanie datasetu

In [8]:
dataset = load_dataset("amu-cai/nEMO", split="train")

### 3. Przetwarzanie próbek audio

In [9]:
# Ustawienie seed dla powtarzalności wyników
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Przetwarzanie danych audio
print("Przetwarzanie próbek audio...")
features = []
labels = []

for sample in tqdm(dataset, desc="Przetwarzanie próbek audio"):
    audio_array = sample['audio']['array']
    sr = sample['audio']['sampling_rate']
    
    # Ujednolicenie długości
    target_length = int(MAX_LENGTH * sr)
    if len(audio_array) > target_length:
        audio_array = audio_array[:target_length]
    else:
        padding = np.zeros(target_length - len(audio_array))
        audio_array = np.concatenate([audio_array, padding])
    
    # Ekstrakcja melspektrogramu
    S = librosa.feature.melspectrogram(y=audio_array, sr=sr, n_mels=128)
    S_db = librosa.power_to_db(S, ref=np.max)
    features.append(S_db)
    
    # Dodanie etykiety
    labels.append(sample['emotion'])


Przetwarzanie próbek audio...


Przetwarzanie próbek audio: 100%|██████████████████████████████████████████████████| 4481/4481 [01:43<00:00, 43.38it/s]


#### Konwersja i normalizacja danych audio oraz etykiet

In [10]:
# Konwersja list na tablice numpy
features = np.array(features)
features = features.reshape(features.shape[0], 1, features.shape[1], features.shape[2])

# Normalizacja danych (standardyzacja)
mean = np.mean(features)
std = np.std(features)
features = (features - mean) / std

# Konwersja etykiet na liczby
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(np.unique(labels))
print(f"Liczba klas emocji: {num_classes}")
print(f"Mapowanie klas: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

Liczba klas emocji: 6
Mapowanie klas: {np.str_('anger'): np.int64(0), np.str_('fear'): np.int64(1), np.str_('happiness'): np.int64(2), np.str_('neutral'): np.int64(3), np.str_('sadness'): np.int64(4), np.str_('surprised'): np.int64(5)}


### 4. Podział danych na zbiory treningowe, walidacyjne i testowe


In [11]:
# Podział na zbiory
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=SEED, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train)


### 5. Inicjalizacja modelu i optymalizatora

In [12]:
# Inicjalizacja modelu, funkcji straty i optymalizatora
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Używane urządzenie: {device}")

# Inicjalizacja modelu
model = AudioResNet(num_classes=num_classes, dropout_rate=0.5)
model = model.to(device)

# Inicjalizacja funkcji straty i optymalizatora
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Dodanie regularyzacji L2
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

Używane urządzenie: cpu


 #### Przygotowanie zestawów danych z augmentacją i ładowanie ich do DataLoaderów

In [13]:
# Przygotowanie zestawów danych z augmentacją
train_dataset = AugmentedAudioDataset(X_train, y_train, augment=True)
val_dataset = AugmentedAudioDataset(X_val, y_val, augment=False)
test_dataset = AugmentedAudioDataset(X_test, y_test, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Ścieżka do zapisywania modelu
early_stopping = EarlyStopping(patience=EARLY_STOPPING_PATIENCE, path=MODEL_PATH)

# Śledzenie historii treningu
history = {
    'train_loss': [],
    'val_loss': [],
    'val_accuracy': []
}

### 6. Pętla treningowa i walidacja modelu


In [14]:
#Pętla treningowa
print("Rozpoczynanie treningu...")
for epoch in range(NUM_EPOCHS):
    # Faza treningu
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)
    history['train_loss'].append(train_loss)
    
    # Faza walidacji
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            # Obliczanie straty walidacyjnej
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Obliczanie dokładności
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)
    
    print(f'Epoka {epoch + 1}/{NUM_EPOCHS}, Strata treningu: {train_loss:.4f}, '
          f'Strata walidacji: {val_loss:.4f}, Dokładność walidacji: {val_accuracy:.2f}%')
    
    # Aktualizacja schedulera
    scheduler.step(val_loss)
    
    # Sprawdzenie warunku early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping aktywowane!")
        break


Rozpoczynanie treningu...
Epoka 1/50, Strata treningu: 1.9047, Strata walidacji: 3.0729, Dokładność walidacji: 34.31%
Validation loss decreased. Saving model to model_outputs\best_model_20250420_115338.pt
Epoka 2/50, Strata treningu: 1.3405, Strata walidacji: 1.0345, Dokładność walidacji: 61.79%
Validation loss decreased. Saving model to model_outputs\best_model_20250420_115338.pt
Epoka 3/50, Strata treningu: 1.0259, Strata walidacji: 1.1134, Dokładność walidacji: 62.06%
EarlyStopping counter: 1 out of 7
Epoka 4/50, Strata treningu: 0.8949, Strata walidacji: 2.0418, Dokładność walidacji: 44.63%
EarlyStopping counter: 2 out of 7
Epoka 5/50, Strata treningu: 0.7370, Strata walidacji: 0.8845, Dokładność walidacji: 68.76%
Validation loss decreased. Saving model to model_outputs\best_model_20250420_115338.pt
Epoka 6/50, Strata treningu: 0.5810, Strata walidacji: 0.9535, Dokładność walidacji: 69.60%
EarlyStopping counter: 1 out of 7
Epoka 7/50, Strata treningu: 0.5733, Strata walidacji: 0.56

### 7. Ewaluacja na zbiorze testowym

In [15]:
# Wczytanie najlepszego modelu
model.load_state_dict(torch.load(MODEL_PATH))

# Testowanie modelu
model.eval()
all_preds = []
all_labels = []
test_correct = 0
test_total = 0
test_loss = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        # Obliczanie straty testowej
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Obliczanie dokładności
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        
        # Zapisanie predykcji i rzeczywistych etykiet
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = test_loss / len(test_loader)
test_accuracy = 100 * test_correct / test_total
print(f'Dokładność testu: {test_accuracy:.2f}%, Strata testu: {test_loss:.4f}')

Dokładność testu: 93.09%, Strata testu: 0.2603


#### Raport klasyfikacji

In [18]:
report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)
report_df.to_csv(os.path.join(MODEL_DIR, f'classification_report_{TIMESTAMP}.csv'))

              precision    recall  f1-score     support
anger          0.959184  0.940000  0.949495  150.000000
fear           0.953333  0.972789  0.962963  147.000000
happiness      0.873333  0.873333  0.873333  150.000000
neutral        0.945455  0.962963  0.954128  162.000000
sadness        0.980392  0.974026  0.977199  154.000000
surprised      0.863636  0.850746  0.857143  134.000000
accuracy       0.930881  0.930881  0.930881    0.930881
macro avg      0.929222  0.928976  0.929044  897.000000
weighted avg   0.930757  0.930881  0.930763  897.000000


## ATAK ADWERSALNY

### Wykorzystaną metodą ataku adwersalnego jest Fast Gradient Sign Method (FGSM).
Jest to jedna z najprostszych i najszybszych metod. FGSM modyfikuje wejście tak, aby maksymalizować stratę, czyli wprowadza model w błąd.  
Etapy ataku:
1. Obliczamy gradient funkcji straty względem danych wejściowych.
2. Tworzymy perturbację (zakłócenie) poprzez pomnożenie znaku gradientu przez niski parametr ε.
3. Dodajemy perturbację do oryginalnego obrazu.


### Przygotowanie funkcji do zastosowania ataku adwersalnego na utworzony model

In [21]:
def fgsm_attack(model, data, target, epsilon, device, criterion):
    """
    Implementacja ataku Fast Gradient Sign Method (FGSM)
    
    Parametry:
    - model: model docelowy do ataku
    - data: tensor wejściowy (melspektrogramy audio)
    - target: prawdziwa etykieta
    - epsilon: współczynnik siły ataku
    - device: urządzenie (CPU/GPU)
    - criterion: funkcja straty
    
    Zwraca:
    - zaburzony przykład (przykład adwersalny)
    """
    # Ustawienie danych jako wymagających gradientu
    data.requires_grad = True
    
    # Forward pass
    output = model(data)
    
    # Obliczenie straty
    loss = criterion(output, target)
    
    # Zerowanie wszystkich poprzednich gradientów
    model.zero_grad()
    
    # Obliczenie gradientów straty względem danych wejściowych
    loss.backward()
    
    # Pobranie znaku gradientu dla uzyskania kierunku
    data_grad = data.grad.data.sign()
    
    # Tworzenie przykładu adwersalnego
    perturbed_data = data + epsilon * data_grad
    
    # Dodanie clippingu, aby dane pozostały w prawidłowym zakresie (zazwyczaj [0,1] lub [-1,1])
    # W zależności od zakresu danych możesz dostosować te wartości
    perturbed_data = torch.clamp(perturbed_data, -3, 3)  # Zakładam znormalizowane dane ~ N(0,1)
    
    return perturbed_data

def test_fgsm(model, test_loader, epsilons, device, criterion, label_encoder, model_dir):
    """
    Testuje skuteczność ataku FGSM dla różnych wartości epsilon
    
    Parametry:
    - model: model docelowy do ataku
    - test_loader: DataLoader zawierający zbiór testowy
    - epsilons: lista wartości epsilon do przetestowania
    - device: urządzenie (CPU/GPU)
    - criterion: funkcja straty
    - label_encoder: enkoder etykiet
    - model_dir: katalog do zapisywania wyników
    """
    # Przełączenie modelu w tryb ewaluacji
    model.eval()
    
    # Przygotowanie list do przechowywania wyników
    accuracies = []
    examples = []
    
    # Testowanie dla każdej wartości epsilon
    for eps in epsilons:
        print(f"\nTestowanie dla epsilon = {eps}")
        correct = 0
        adv_examples = []
        batch_count = 0
        
        for data, target in tqdm(test_loader, desc=f"Atak FGSM (eps={eps})"):
            data, target = data.to(device), target.to(device)
            
            # Przeprowadzenie ataku FGSM
            perturbed_data = fgsm_attack(model, data.clone(), target, eps, device, criterion)
            
            # Predykcja na przykładach adwersalnych
            with torch.no_grad():
                output = model(perturbed_data)
            
            # Wyznaczenie predykcji
            _, final_pred = torch.max(output.data, 1)
            
            # Zliczenie poprawnych klasyfikacji (tzn. takich, które nie zostały zmylone przez atak)
            correct += (final_pred == target).sum().item()
            
            # Zapisanie kilku przykładów do wizualizacji
            if batch_count < 5:
                for i in range(min(len(data), 3)):  # Zapisz max 3 przykłady z każdej partii
                    if len(adv_examples) < 10 and eps > 0:  # Ogranicz do 10 przykładów
                        orig = data[i].detach().cpu().numpy()
                        pert = perturbed_data[i].detach().cpu().numpy()
                        adv_pred = final_pred[i].item()
                        orig_pred = target[i].item()
                        adv_examples.append((orig, pert, orig_pred, adv_pred))
            batch_count += 1
        
        # Obliczenie dokładności
        final_acc = correct / len(test_loader.dataset)
        print(f"Dokładność po ataku: {final_acc * 100:.2f}%")
        
        # Zapisanie wyników
        accuracies.append(final_acc)
        examples.append(adv_examples)
    
    # Wizualizacja wyników
    plt.figure(figsize=(10, 6))
    plt.plot(epsilons, accuracies, "*-")
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.xticks(epsilons)
    plt.xlabel("Epsilon")
    plt.ylabel("Dokładność")
    plt.title("Dokładność modelu pod wpływem ataku FGSM")
    plt.grid(True)
    plt.savefig(os.path.join(model_dir, "fgsm_accuracy_plot.png"))
    plt.close()
    
    # Wizualizacja przykładów adwersalnych (dla melspektrogramów)
    visualize_adversarial_examples(examples, epsilons, label_encoder, model_dir)
    
    return accuracies, examples

def visualize_adversarial_examples(examples, epsilons, label_encoder, model_dir):
    """
    Wizualizuje przykłady adwersalne dla melspektrogramów
    
    Parametry:
    - examples: lista przykładów adwersalnych
    - epsilons: lista wartości epsilon
    - label_encoder: enkoder etykiet
    - model_dir: katalog do zapisywania wyników
    """
    class_names = label_encoder.classes_
    
    # Dla każdej wartości epsilon (oprócz 0)
    cnt = 0
    for i, eps in enumerate(epsilons):
        if eps == 0 or not examples[i]:
            continue
            
        # Wybierz maksymalnie 5 przykładów
        for j in range(min(len(examples[i]), 5)):
            cnt += 1
            orig, adv, orig_label, adv_label = examples[i][j]
            
            # Zamiana z formatu tensora (1, height, width) na format obrazu (height, width)
            orig = orig.squeeze(0)
            adv = adv.squeeze(0)
            
            # Obliczenie perturbacji (różnicy)
            diff = adv - orig
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Oryginalny melspektrogram
            im0 = axes[0].imshow(orig, aspect='auto', origin='lower', cmap='viridis')
            axes[0].set_title(f"Oryginalny\nKlasa: {class_names[orig_label]}")
            axes[0].set_ylabel("Mel Bins")
            axes[0].set_xlabel("Frames")
            
            # Perturbacja (różnica)
            im1 = axes[1].imshow(diff, aspect='auto', origin='lower', cmap='coolwarm')
            axes[1].set_title(f"Perturbacja\nEpsilon: {eps}")
            axes[1].set_xlabel("Frames")
            
            # Przykład adwersalny
            im2 = axes[2].imshow(adv, aspect='auto', origin='lower', cmap='viridis')
            axes[2].set_title(f"Adwersalny\nKlasa: {class_names[adv_label]}")
            axes[2].set_xlabel("Frames")
            
            # Dodaj paski kolorów
            plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)
            plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
            plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
            
            plt.tight_layout()
            plt.savefig(os.path.join(model_dir, f"adversarial_example_{eps}_{j}.png"))
            plt.close()
            
            if cnt >= 15:  # Ograniczenie do 15 przykładów
                return

# Funkcja do przeprowadzenia całego eksperymentu
def run_fgsm_experiment(model, test_loader, device, criterion, label_encoder, model_dir):
    """
    Przeprowadza pełny eksperyment z atakiem FGSM
    
    Parametry:
    - model: model docelowy do ataku
    - test_loader: DataLoader zawierający zbiór testowy
    - device: urządzenie (CPU/GPU)
    - criterion: funkcja straty
    - label_encoder: enkoder etykiet
    - model_dir: katalog do zapisywania wyników
    """
    print("Rozpoczynam eksperyment z atakiem FGSM...")
    
    # Testowanie modelu na oryginalnych (niezakłóconych) danych
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Ewaluacja na oryginalnych danych"):
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, pred = torch.max(output.data, 1)
            correct += (pred == target).sum().item()
    
    baseline_acc = correct / len(test_loader.dataset)
    print(f"Bazowa dokładność (bez ataku): {baseline_acc * 100:.2f}%")
    
    # Wartości epsilon do przetestowania
    epsilons = [0, 0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]
    
    # Przeprowadzenie ataku FGSM dla różnych wartości epsilon
    accuracies, examples = test_fgsm(model, test_loader, epsilons, device, criterion, label_encoder, model_dir)
    
    # Zapisanie wyników do pliku CSV
    import pandas as pd
    results = pd.DataFrame({
        'epsilon': epsilons,
        'accuracy': accuracies
    })
    results.to_csv(os.path.join(model_dir, "fgsm_results.csv"), index=False)
    
    print("\nEksperyment z atakiem FGSM zakończony.")
    print(f"Wyniki zostały zapisane w katalogu: {model_dir}")
    
    return results


#### Zapisanie wyników i wizualizacja ataku adwersalnego

In [24]:
# Utworzenie katalogu dla wyników ataku FGSM
FGSM_DIR = os.path.join(MODEL_DIR, f'fgsm_results_{TIMESTAMP}')
os.makedirs(FGSM_DIR, exist_ok=True)

print("Rozpoczynam testowanie odporności modelu na atak FGSM...")

# Uruchomienie eksperymentu z atakiem FGSM
results = run_fgsm_experiment(model, test_loader, device, criterion, label_encoder, FGSM_DIR)

# Wyświetlenie tabeli wyników
print("\nWyniki ataku FGSM:")
print(results)

# Analiza najbardziej podatnych klas na atak dla wielu wartości epsilon
print("\nAnalizuję podatność poszczególnych klas na atak FGSM dla różnych wartości epsilon...")

# Lista wartości epsilon do analizy podatności klas
target_epsilons = [0.001, 0.005, 0.01]  

# Słownik do przechowywania wyników dla każdej wartości epsilon
class_accuracy_results = {}

# Dla każdej wartości epsilon wykonaj analizę
for target_epsilon in target_epsilons:
    print(f"\nAnaliza dla epsilon = {target_epsilon}")
    
    # Przechowywanie macierzy konfuzji dla ataku
    all_preds_adv = []
    all_labels_adv = []
    
    # Uruchomienie ataku z wybraną wartością epsilon
    model.eval()
    for data, target in tqdm(test_loader, desc=f"Atak FGSM (eps={target_epsilon})"):
        data, target = data.to(device), target.to(device)
        
        # Przeprowadzenie ataku FGSM
        perturbed_data = fgsm_attack(model, data.clone(), target, target_epsilon, device, criterion)
        
        # Predykcja na przykładach adwersalnych
        with torch.no_grad():
            output = model(perturbed_data)
        
        # Wyznaczenie predykcji
        _, predicted = torch.max(output.data, 1)
        
        # Zapisanie predykcji i rzeczywistych etykiet
        all_preds_adv.extend(predicted.cpu().numpy())
        all_labels_adv.extend(target.cpu().numpy())
    
    # Macierz konfuzji dla ataku
    cm_adv = confusion_matrix(all_labels_adv, all_preds_adv)
    class_names = label_encoder.classes_
    
    # Wizualizacja macierzy konfuzji dla ataku
    plt.figure(figsize=(10, 8))
    cm_normalized_adv = cm_adv.astype('float') / cm_adv.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized_adv, annot=True, fmt='.2f', cmap='Reds', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Znormalizowana macierz konfuzji po ataku FGSM (ε={target_epsilon})')
    plt.ylabel('Rzeczywista etykieta')
    plt.xlabel('Przewidziana etykieta')
    plt.tight_layout()
    plt.savefig(os.path.join(FGSM_DIR, f'confusion_matrix_fgsm_{target_epsilon}.png'))
    plt.close()
    
    # Raport klasyfikacji dla ataku
    report_adv = classification_report(all_labels_adv, all_preds_adv, target_names=class_names, output_dict=True)
    report_df_adv = pd.DataFrame(report_adv).transpose()
    print(f"\nRaport klasyfikacji po ataku FGSM (epsilon={target_epsilon}):")
    print(report_df_adv)
    report_df_adv.to_csv(os.path.join(FGSM_DIR, f'classification_report_fgsm_{target_epsilon}.csv'))
    
    # Zapisanie dokładności dla każdej klasy przy danej wartości epsilon
    class_accuracies = {name: report_adv[name]['recall'] for name in class_names}
    class_accuracy_results[target_epsilon] = class_accuracies

# Utworzenie DataFrame z wynikami dla wszystkich wartości epsilon
comparison_df = pd.DataFrame(index=class_names)

# Bazowa dokładność (bez ataku)
baseline_accuracies = {name: report[name]['recall'] for name in class_names}
comparison_df['Dokładność bazowa'] = pd.Series(baseline_accuracies)

# Dodanie kolumn dla każdej wartości epsilon
for eps in target_epsilons:
    comparison_df[f'Dokładność (ε={eps})'] = pd.Series(class_accuracy_results[eps])
    comparison_df[f'Spadek (ε={eps})'] = comparison_df['Dokładność bazowa'] - comparison_df[f'Dokładność (ε={eps})']

# Sortowanie według średniego spadku dokładności
comparison_df['Średni spadek'] = comparison_df[[f'Spadek (ε={eps})' for eps in target_epsilons]].mean(axis=1)
comparison_df = comparison_df.sort_values('Średni spadek', ascending=False)

print("\nPorównanie dokładności klas przed i po ataku FGSM dla różnych wartości epsilon:")
print(comparison_df)
comparison_df.to_csv(os.path.join(FGSM_DIR, 'accuracy_comparison_all_epsilons.csv'))

# Wizualizacja spadku dokładności dla każdej klasy w funkcji epsilon
plt.figure(figsize=(12, 8))
for class_name in class_names:
    accuracies = [baseline_accuracies[class_name]] + [class_accuracy_results[eps][class_name] for eps in target_epsilons]
    plt.plot([0] + target_epsilons, accuracies, marker='o', linewidth=2, label=class_name)

plt.xlabel('Epsilon (siła ataku)')
plt.ylabel('Dokładność klasyfikacji')
plt.title('Spadek dokładności klasyfikacji emocji pod wpływem ataku FGSM')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.05)
plt.savefig(os.path.join(FGSM_DIR, 'accuracy_vs_epsilon_by_class.png'))
plt.close()

print(f"\nAnaliza zakończona. Wszystkie wyniki zostały zapisane w katalogu: {FGSM_DIR}")

Rozpoczynam testowanie odporności modelu na atak FGSM...
Rozpoczynam eksperyment z atakiem FGSM...


Ewaluacja na oryginalnych danych: 100%|████████████████████████████████████████████████| 29/29 [00:45<00:00,  1.56s/it]


Bazowa dokładność (bez ataku): 93.09%

Testowanie dla epsilon = 0


Atak FGSM (eps=0): 100%|███████████████████████████████████████████████████████████████| 29/29 [03:05<00:00,  6.40s/it]


Dokładność po ataku: 93.09%

Testowanie dla epsilon = 0.001


...


Raport klasyfikacji po ataku FGSM (epsilon=0.001):
              precision    recall  f1-score     support
anger          0.945578  0.926667  0.936027  150.000000
fear           0.934641  0.972789  0.953333  147.000000
happiness      0.851351  0.840000  0.845638  150.000000
neutral        0.928571  0.962963  0.945455  162.000000
sadness        0.980132  0.961039  0.970492  154.000000
surprised      0.830769  0.805970  0.818182  134.000000
accuracy       0.914158  0.914158  0.914158    0.914158
macro avg      0.911841  0.911571  0.911521  897.000000
weighted avg   0.913739  0.914158  0.913763  897.000000

Analiza dla epsilon = 0.005


Atak FGSM (eps=0.005): 100%|███████████████████████████████████████████████████████████| 29/29 [02:06<00:00,  4.35s/it]



Raport klasyfikacji po ataku FGSM (epsilon=0.005):
              precision    recall  f1-score     support
anger          0.895522  0.800000  0.845070  150.000000
fear           0.883117  0.925170  0.903654  147.000000
happiness      0.700000  0.746667  0.722581  150.000000
neutral        0.842105  0.888889  0.864865  162.000000
sadness        0.933775  0.915584  0.924590  154.000000
surprised      0.645669  0.611940  0.628352  134.000000
accuracy       0.819398  0.819398  0.819398    0.819398
macro avg      0.816698  0.814708  0.814852  897.000000
weighted avg   0.820389  0.819398  0.819040  897.000000

Analiza dla epsilon = 0.01


Atak FGSM (eps=0.01): 100%|████████████████████████████████████████████████████████████| 29/29 [02:06<00:00,  4.36s/it]



Raport klasyfikacji po ataku FGSM (epsilon=0.01):
              precision    recall  f1-score     support
anger          0.813008  0.666667  0.732601  150.000000
fear           0.783439  0.836735  0.809211  147.000000
happiness      0.522727  0.613333  0.564417  150.000000
neutral        0.685185  0.685185  0.685185  162.000000
sadness        0.813333  0.792208  0.802632  154.000000
surprised      0.379845  0.365672  0.372624  134.000000
accuracy       0.665552  0.665552  0.665552    0.665552
macro avg      0.666256  0.659967  0.661111  897.000000
weighted avg   0.671882  0.665552  0.666715  897.000000

Porównanie dokładności klas przed i po ataku FGSM dla różnych wartości epsilon:
           Dokładność bazowa  Dokładność (ε=0.001)  Spadek (ε=0.001)  \
surprised           0.850746              0.805970          0.044776   
anger               0.940000              0.926667          0.013333   
happiness           0.873333              0.840000          0.033333   
neutral             

### PODSUMOWANIE
#### Przy wartości epsilon = 0.01 dokładność modelu znacznie spada; do 66.56%. Od wartości epsilon = 0.05 dokładność modelu oscyluje w graniach kilku procent. Najbardziej podatną na wprowadzane zakłócenia emocją jest zaskoczenie (największy spadek w klasyfikowaniu tej emocji), a najmniej strach.  